### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
!pip install langchain

  Using cached langchain-0.3.18-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_text_splitters-0.3.6-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.3.8-py3-none-any.whl.metadata (14 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached aiohappyeyeballs-2.4.6-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.1.0-cp310-cp310-win_amd64.whl.metadata (5.1 kB)
  Using cached propcache-0.2.1-cp310-cp310-win_amd64.whl.metadata (9.5 kB)
  Using cached yarl-1.18.3-cp310-cp310-win_amd64.whl.metadata (71 kB)
  Using cached jsonpa

In [3]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")


groq_api_key=os.getenv("GROQ_API_KEY")
# groq_api_key

In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C7CE3A1CC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C7CE3A3070>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [23]:
from langchain_core.messages import SystemMessage,HumanMessage

messages=[
    SystemMessage(content="translate following from english to french"),
    HumanMessage(content="hey!! i am harshit")
]
result=model.invoke(messages)

In [24]:
result

AIMessage(content='Hey!! Je suis Harshit. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 21, 'total_tokens': 32, 'completion_time': 0.02, 'prompt_time': 0.001077157, 'queue_time': 0.023102481, 'total_time': 0.021077157}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-cd458272-9636-46ce-8bd4-1c42ec536b60-0', usage_metadata={'input_tokens': 21, 'output_tokens': 11, 'total_tokens': 32})

In [25]:
result.content

'Hey!! Je suis Harshit. \n'

In [26]:
from langchain_core.output_parsers import StrOutputParser
parcer=StrOutputParser()
parcer.invoke(result)

'Hey!! Je suis Harshit. \n'

In [27]:
## Using LCEL chain component
chain=model|parcer
chain.invoke(messages)

'Hé ! Je suis Harshit. \n'

In [28]:
## Prompt templates
from langchain_core.prompts import ChatPromptTemplate
template="transltate message to {language}"

prompt=ChatPromptTemplate.from_messages([
    ('system',template),
    ("user","{text}")
])


In [32]:
result=prompt.invoke({"language":"hindi","text":"Hiii"})

In [33]:
result.messages

[SystemMessage(content='transltate message to hindi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hiii', additional_kwargs={}, response_metadata={})]

In [34]:
##Chaining together components with LCEL
chain=prompt|model|parcer

In [45]:
chain.invoke({"language":"hindi","text":"who u"})

'Here\'s how you can translate "Who u" to Hindi:\n\n**Informal:**\n\n* **तुम कौन हो?** (Tum kaun ho?) - This is the most common and direct way to ask "Who are you?".\n\n**Slightly more formal:**\n\n* **आप कौन हैं?** (Aap kaun hain?) \n\n\nLet me know if you have any other phrases you\'d like translated!\n'